In [1]:
import numpy as np
import pandas as pd
from mpl_toolkits.mplot3d import Axes3D
import matplotlib.pyplot as plt
import tables
from random import shuffle
from IPython.display import clear_output
from sklearn import metrics
from tqdm import tqdm
import time
import seaborn as sns
%matplotlib inline

from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets

/storage/home/vprov/anaconda3/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/storage/home/vprov/anaconda3/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/storage/home/vprov/anaconda3/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/storage/home/vprov/anaconda3/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/storage/home/vprov/anaconda3/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/storage/h

In [2]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

use_cuda = torch.cuda.is_available()
Tensor = FloatTensor = torch.cuda.FloatTensor if use_cuda else torch.FloatTensor
LongTensor = torch.cuda.LongTensor if use_cuda else torch.LongTensor
ByteTensor = torch.cuda.ByteTensor if use_cuda else torch.ByteTensor

In [3]:
DATA_DIR = '/storage/home/vprov/JUNO/JUNO_students/data/'
lpmt_hits = pd.read_hdf(DATA_DIR + 'train/train_lpmt_hits.h5', mode='r') 
lpmt_hits.shape

(92717003, 4)

In [4]:
spmt_hits = pd.read_hdf(DATA_DIR + 'train/train_spmt_hits.h5', mode='r')
lpmt_pos = pd.read_csv(DATA_DIR + 'train/lpmt_pos.csv') 
spmt_pos = pd.read_csv(DATA_DIR + 'train/spmt_pos.csv') 

In [14]:
train_info = pd.read_csv(DATA_DIR + 'train/train_info.csv')

In [31]:
len(lpmt_hits['event'].unique())

10000

In [6]:
total_pos = lpmt_pos.append(spmt_pos) 

In [7]:
len(total_pos)

42691

In [8]:
total_pos[:5]

,pmt_id,pmt_x,pmt_y,pmt_z
0,0,1.049021e+03,0.0000,19171.32
1,1,9.084785e+02,524.5103,19171.32
2,2,5.245103e+02,908.4785,19171.32
3,3,6.423399e-14,1049.0206,19171.32
4,4,-5.245103e+02,908.4785,19171.32


In [9]:
total_pos[-5:]

,pmt_id,pmt_x,pmt_y,pmt_z
24947,336567,3.692840e-14,603.08655,-19190.525
24948,336568,-5.222883e+02,301.54327,-19190.525
24949,336569,-5.222883e+02,-301.54327,-19190.525
24950,336570,-1.107852e-13,-603.08655,-19190.525
24951,336571,5.222883e+02,-301.54327,-19190.525


In [10]:
lpmt_hits[:5]

,event,hitTime,isDN,pmtID
0,0,249.992615,False,14175
1,0,40.010311,False,17319
2,0,162.123199,False,16882
3,0,51.875614,False,14951
4,0,79.817497,False,10947


In [59]:
lpmt_n_hits = pd.read_csv(DATA_DIR + 'train/train_lpmt_n_hits.csv').values[:,1]
spmt_n_hits = pd.read_csv(DATA_DIR + 'train/train_spmt_n_hits.csv').values[:,1]

In [60]:
lpmt_n_hits[:3]

array([8902, 7305, 6718])

In [61]:
len(lpmt_n_hits)

10000

In [62]:
len(lpmt_hits[lpmt_hits['event'] == 0])

8902

In [63]:
len(lpmt_hits[lpmt_hits['event'] == 1])

7305

In [64]:
lpmt_hits[:3]

,event,hitTime,isDN,pmtID
0,0,249.992615,False,14175
1,0,40.010311,False,17319
2,0,162.123199,False,16882


In [65]:
max_events = 100 

## Noise classification

**Preprocessing from baseline**

In [72]:
def generate_hit_feature(f):
    # f-function
    lpmt_result = np.zeros(len(lpmt_hits))
    spmt_result = np.zeros(len(spmt_hits))
    total_lpmt_hits = 0
    total_spmt_hits = 0

    for event_id in tqdm(range(max_events)):
        # Number of detections on current id
        lnh = int(lpmt_n_hits[event_id])
        snh = int(spmt_n_hits[event_id])
        
        # Collect all current events info event, hitTime, isDN, pmtID
        event_hits = lpmt_hits[total_lpmt_hits:total_lpmt_hits+lnh].append(
                     spmt_hits[total_spmt_hits:total_spmt_hits+snh])
        
        values = np.array(f(event_hits))
        
        lpmt_result[total_lpmt_hits:total_lpmt_hits+lnh] = values[:lnh]
        spmt_result[total_spmt_hits:total_spmt_hits+snh] = values[lnh:]
    
        total_lpmt_hits += lnh
        total_spmt_hits += snh
    
    return lpmt_result, spmt_result

In [67]:
total_pos[:3]

,pmt_id,pmt_x,pmt_y,pmt_z
0,0,1049.0206,0.0000,19171.32
1,1,908.4785,524.5103,19171.32
2,2,524.5103,908.4785,19171.32


In [41]:
neibours = np.zeros((len(total_pos), 100), dtype='int32')

coord_list = []
for _, x, y, z in total_pos.itertuples(index=False):
    coord_list.append((x, y, z))

coord = np.array(coord_list)

for i, (id1, x1, y1, z1) in tqdm(enumerate(total_pos.itertuples(index=False)), total=len(total_pos)):
    dists = np.sum(np.square(coord - (x1, y1, z1)), axis=1)
    neibours[i] = np.argsort(dists)[1:101]

100%|██████████| 42691/42691 [03:08<00:00, 225.88it/s]


In [68]:
# dict from pmt_id to its number in total_pos
pmt_id_map = {v:i for i, v in enumerate(total_pos['pmt_id'].values)}

In [73]:
def count_neibours(top_n, dtime):
    def inner(event_hits):
        d = {}
        # event, hitTime, idDN, pmtID
        for _, time, _, mtID in event_hits.itertuples(index=False):
            if (mtID in pmt_id_map):
                i = pmt_id_map[mtID]
                l = d.get(i, [])
                l.append(time)
                d[i] = l

        d = {k: np.array(v) for k, v in d.items()}

        values = []
        for _, time, _, mtID in event_hits.itertuples(index=False):
            if (mtID not in pmt_id_map): # If it is new id then append 0
                values.append(0)
                continue
            # Number in total_pos index
            i = pmt_id_map[mtID]
            value = 0 
            # For nearest pmts 
            for j in neibours[i,0:top_n]:
                if j in d:
                    # Our time - neighbour time
                    value += np.sum(np.abs(time - d[j]) < dtime)
            values.append(value)
        
        return values
    return inner

In [74]:
def count_hits_in_event(event_hits):
    d = {}
    for _, time, _, mtID in event_hits.itertuples(index=False):
        d[mtID] = d.get(mtID, 0) + 1
        
    values = []
    for _, time, _, mtID in event_hits.itertuples(index=False):
        values.append(d[mtID])
            
    return values

In [75]:
lpmt_f1, spmt_f1 = generate_hit_feature(count_hits_in_event)

100%|██████████| 100/100 [00:02<00:00, 48.06it/s]


In [76]:
lpmt_f2, spmt_f2 = generate_hit_feature(count_neibours(20, 100))

100%|██████████| 100/100 [01:14<00:00,  1.60it/s]


In [77]:
lpmt_f3, spmt_f3 = generate_hit_feature(count_neibours(60, 100))

100%|██████████| 100/100 [03:49<00:00,  1.96s/it]


In [78]:
lpmt_f1

array([ 2., 12.,  2., ...,  0.,  0.,  0.])

In [82]:
len(lpmt_f1)

92717003

In [79]:
spmt_f1

array([1., 1., 1., ..., 0., 0., 0.])

In [80]:
lpmt_f2

array([ 0., 36.,  4., ...,  0.,  0.,  0.])

In [81]:
spmt_f2

array([2., 5., 2., ..., 0., 0., 0.])

In [83]:
X_lpmt = np.stack([lpmt_f1, lpmt_f2, lpmt_f3], axis=1)
y_lpmt = lpmt_hits['isDN'].values

In [84]:
X_lpmt = X_lpmt[lpmt_hits['event'] < max_events]
y_lpmt = y_lpmt[lpmt_hits['event'] < max_events]

In [85]:
X_spmt = np.stack([spmt_f1, spmt_f2, spmt_f3], axis=1)
y_spmt = spmt_hits['isDN'].values

In [86]:
X_spmt = X_spmt[spmt_hits['event'] < max_events]
y_spmt = y_spmt[spmt_hits['event'] < max_events]

In [94]:
len(X_spmt)

16136

In [89]:
train_info[:3]

,E,R,evtID,x,y,z
0,4.747791,14610.378,0,8290.7790,11995.6180,911.74286
1,3.919721,14630.141,1,11397.6320,5407.4497,-7409.08200
2,3.765940,16820.080,3,-2377.9307,-16317.7020,3315.59030


In [91]:
lpmt_energy = lpmt_hits.merge(train_info, left_on='event', right_on='evtID')

In [92]:
lpmt_energy[:5]

,event,hitTime,isDN,pmtID,E,R,evtID,x,y,z
0,0,249.992615,False,14175,4.747791,14610.378,0,8290.779,11995.618,911.74286
1,0,40.010311,False,17319,4.747791,14610.378,0,8290.779,11995.618,911.74286
2,0,162.123199,False,16882,4.747791,14610.378,0,8290.779,11995.618,911.74286
3,0,51.875614,False,14951,4.747791,14610.378,0,8290.779,11995.618,911.74286
4,0,79.817497,False,10947,4.747791,14610.378,0,8290.779,11995.618,911.74286


In [93]:
lpmt_energy.isnull().any()

event      False
hitTime    False
isDN       False
pmtID      False
E          False
R          False
evtID      False
x          False
y          False
z          False
dtype: bool

In [97]:
import catboost

In [96]:
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score

In [98]:
from catboost import CatBoostClassifier

In [116]:
y_int = y_lpmt.astype(int)

In [121]:
len(y_int)

643696

In [122]:
np.sum(y_int)

51740

In [123]:
(len(y_int) - np.sum(y_int)) / np.sum(y_int)

11.440974101275609

In [127]:
catclass = CatBoostClassifier()#class_weights=[1.,11.])

In [128]:
catclass.fit(X_lpmt, y_int)

0:	learn: 0.5109212	total: 58.9ms	remaining: 58.9s
1:	learn: 0.3980850	total: 120ms	remaining: 1m
2:	learn: 0.3222423	total: 176ms	remaining: 58.6s
3:	learn: 0.2745379	total: 234ms	remaining: 58.3s
4:	learn: 0.2419136	total: 294ms	remaining: 58.6s
5:	learn: 0.2192119	total: 352ms	remaining: 58.3s
6:	learn: 0.2006686	total: 409ms	remaining: 58.1s
7:	learn: 0.1872417	total: 467ms	remaining: 57.9s
8:	learn: 0.1791581	total: 527ms	remaining: 58.1s
9:	learn: 0.1726287	total: 586ms	remaining: 58s
10:	learn: 0.1682589	total: 644ms	remaining: 57.9s
11:	learn: 0.1642644	total: 702ms	remaining: 57.8s
12:	learn: 0.1615212	total: 764ms	remaining: 58s
13:	learn: 0.1595714	total: 823ms	remaining: 57.9s
14:	learn: 0.1578794	total: 880ms	remaining: 57.8s
15:	learn: 0.1564215	total: 941ms	remaining: 57.9s
16:	learn: 0.1552916	total: 1000ms	remaining: 57.8s
17:	learn: 0.1548475	total: 1.06s	remaining: 57.6s
18:	learn: 0.1542675	total: 1.11s	remaining: 57.5s
19:	learn: 0.1537724	total: 1.17s	remaining: 5

163:	learn: 0.1507988	total: 9.57s	remaining: 48.8s
164:	learn: 0.1507982	total: 9.63s	remaining: 48.8s
165:	learn: 0.1507976	total: 9.69s	remaining: 48.7s
166:	learn: 0.1507974	total: 9.75s	remaining: 48.6s
167:	learn: 0.1507972	total: 9.81s	remaining: 48.6s
168:	learn: 0.1507969	total: 9.87s	remaining: 48.5s
169:	learn: 0.1507968	total: 9.93s	remaining: 48.5s
170:	learn: 0.1507951	total: 9.99s	remaining: 48.4s
171:	learn: 0.1507944	total: 10s	remaining: 48.4s
172:	learn: 0.1507939	total: 10.1s	remaining: 48.3s
173:	learn: 0.1507934	total: 10.2s	remaining: 48.2s
174:	learn: 0.1507931	total: 10.2s	remaining: 48.2s
175:	learn: 0.1507927	total: 10.3s	remaining: 48.1s
176:	learn: 0.1507921	total: 10.3s	remaining: 48.1s
177:	learn: 0.1507917	total: 10.4s	remaining: 48s
178:	learn: 0.1507907	total: 10.5s	remaining: 47.9s
179:	learn: 0.1507897	total: 10.5s	remaining: 47.9s
180:	learn: 0.1507890	total: 10.6s	remaining: 47.8s
181:	learn: 0.1507888	total: 10.6s	remaining: 47.8s
182:	learn: 0.15

323:	learn: 0.1506992	total: 18.8s	remaining: 39.3s
324:	learn: 0.1506992	total: 18.9s	remaining: 39.3s
325:	learn: 0.1506991	total: 19s	remaining: 39.2s
326:	learn: 0.1506988	total: 19s	remaining: 39.1s
327:	learn: 0.1506988	total: 19.1s	remaining: 39.1s
328:	learn: 0.1506977	total: 19.1s	remaining: 39s
329:	learn: 0.1506976	total: 19.2s	remaining: 39s
330:	learn: 0.1506974	total: 19.2s	remaining: 38.9s
331:	learn: 0.1506970	total: 19.3s	remaining: 38.9s
332:	learn: 0.1506962	total: 19.4s	remaining: 38.8s
333:	learn: 0.1506961	total: 19.4s	remaining: 38.7s
334:	learn: 0.1506959	total: 19.5s	remaining: 38.7s
335:	learn: 0.1506955	total: 19.5s	remaining: 38.6s
336:	learn: 0.1506955	total: 19.6s	remaining: 38.6s
337:	learn: 0.1506954	total: 19.7s	remaining: 38.5s
338:	learn: 0.1506946	total: 19.7s	remaining: 38.4s
339:	learn: 0.1506944	total: 19.8s	remaining: 38.4s
340:	learn: 0.1506940	total: 19.8s	remaining: 38.3s
341:	learn: 0.1506940	total: 19.9s	remaining: 38.3s
342:	learn: 0.150693

483:	learn: 0.1506366	total: 28.1s	remaining: 30s
484:	learn: 0.1506363	total: 28.2s	remaining: 29.9s
485:	learn: 0.1506363	total: 28.2s	remaining: 29.9s
486:	learn: 0.1506363	total: 28.3s	remaining: 29.8s
487:	learn: 0.1506363	total: 28.4s	remaining: 29.7s
488:	learn: 0.1506363	total: 28.4s	remaining: 29.7s
489:	learn: 0.1506358	total: 28.5s	remaining: 29.6s
490:	learn: 0.1506354	total: 28.5s	remaining: 29.6s
491:	learn: 0.1506352	total: 28.6s	remaining: 29.5s
492:	learn: 0.1506352	total: 28.6s	remaining: 29.5s
493:	learn: 0.1506350	total: 28.7s	remaining: 29.4s
494:	learn: 0.1506349	total: 28.8s	remaining: 29.3s
495:	learn: 0.1506348	total: 28.8s	remaining: 29.3s
496:	learn: 0.1506347	total: 28.9s	remaining: 29.2s
497:	learn: 0.1506347	total: 28.9s	remaining: 29.2s
498:	learn: 0.1506344	total: 29s	remaining: 29.1s
499:	learn: 0.1506339	total: 29s	remaining: 29s
500:	learn: 0.1506339	total: 29.1s	remaining: 29s
501:	learn: 0.1506337	total: 29.2s	remaining: 28.9s
502:	learn: 0.1506336	

643:	learn: 0.1506065	total: 37.4s	remaining: 20.7s
644:	learn: 0.1506065	total: 37.5s	remaining: 20.6s
645:	learn: 0.1506065	total: 37.5s	remaining: 20.6s
646:	learn: 0.1506065	total: 37.6s	remaining: 20.5s
647:	learn: 0.1506062	total: 37.6s	remaining: 20.4s
648:	learn: 0.1506061	total: 37.7s	remaining: 20.4s
649:	learn: 0.1506061	total: 37.7s	remaining: 20.3s
650:	learn: 0.1506058	total: 37.8s	remaining: 20.3s
651:	learn: 0.1506053	total: 37.9s	remaining: 20.2s
652:	learn: 0.1506050	total: 37.9s	remaining: 20.1s
653:	learn: 0.1506045	total: 38s	remaining: 20.1s
654:	learn: 0.1506043	total: 38s	remaining: 20s
655:	learn: 0.1506038	total: 38.1s	remaining: 20s
656:	learn: 0.1506038	total: 38.1s	remaining: 19.9s
657:	learn: 0.1506036	total: 38.2s	remaining: 19.9s
658:	learn: 0.1506036	total: 38.3s	remaining: 19.8s
659:	learn: 0.1506034	total: 38.3s	remaining: 19.7s
660:	learn: 0.1506033	total: 38.4s	remaining: 19.7s
661:	learn: 0.1506033	total: 38.4s	remaining: 19.6s
662:	learn: 0.150603

803:	learn: 0.1505727	total: 46.7s	remaining: 11.4s
804:	learn: 0.1505726	total: 46.8s	remaining: 11.3s
805:	learn: 0.1505726	total: 46.8s	remaining: 11.3s
806:	learn: 0.1505724	total: 46.9s	remaining: 11.2s
807:	learn: 0.1505724	total: 47s	remaining: 11.2s
808:	learn: 0.1505724	total: 47s	remaining: 11.1s
809:	learn: 0.1505723	total: 47.1s	remaining: 11s
810:	learn: 0.1505721	total: 47.1s	remaining: 11s
811:	learn: 0.1505721	total: 47.2s	remaining: 10.9s
812:	learn: 0.1505721	total: 47.3s	remaining: 10.9s
813:	learn: 0.1505721	total: 47.3s	remaining: 10.8s
814:	learn: 0.1505721	total: 47.4s	remaining: 10.8s
815:	learn: 0.1505721	total: 47.4s	remaining: 10.7s
816:	learn: 0.1505720	total: 47.5s	remaining: 10.6s
817:	learn: 0.1505719	total: 47.5s	remaining: 10.6s
818:	learn: 0.1505717	total: 47.6s	remaining: 10.5s
819:	learn: 0.1505716	total: 47.7s	remaining: 10.5s
820:	learn: 0.1505714	total: 47.7s	remaining: 10.4s
821:	learn: 0.1505713	total: 47.8s	remaining: 10.3s
822:	learn: 0.150571

963:	learn: 0.1505406	total: 56s	remaining: 2.09s
964:	learn: 0.1505406	total: 56.1s	remaining: 2.03s
965:	learn: 0.1505405	total: 56.2s	remaining: 1.98s
966:	learn: 0.1505402	total: 56.2s	remaining: 1.92s
967:	learn: 0.1505401	total: 56.3s	remaining: 1.86s
968:	learn: 0.1505399	total: 56.3s	remaining: 1.8s
969:	learn: 0.1505399	total: 56.4s	remaining: 1.74s
970:	learn: 0.1505399	total: 56.4s	remaining: 1.69s
971:	learn: 0.1505383	total: 56.5s	remaining: 1.63s
972:	learn: 0.1505374	total: 56.6s	remaining: 1.57s
973:	learn: 0.1505373	total: 56.6s	remaining: 1.51s
974:	learn: 0.1505373	total: 56.7s	remaining: 1.45s
975:	learn: 0.1505373	total: 56.7s	remaining: 1.4s
976:	learn: 0.1505372	total: 56.8s	remaining: 1.34s
977:	learn: 0.1505372	total: 56.9s	remaining: 1.28s
978:	learn: 0.1505372	total: 56.9s	remaining: 1.22s
979:	learn: 0.1505372	total: 57s	remaining: 1.16s
980:	learn: 0.1505372	total: 57s	remaining: 1.1s
981:	learn: 0.1505372	total: 57.1s	remaining: 1.05s
982:	learn: 0.1505372

In [129]:
accuracy_score(y_int, catclass.predict(X_lpmt))

0.9424914866645124

In [130]:
f1_score(y_int, catclass.predict(X_lpmt))

0.616568611203182

**Now let's divide to train and test**

In [131]:
from sklearn.model_selection import train_test_split

In [132]:
Xl_train, Xl_test, yl_train, yl_test = train_test_split(X_lpmt, y_int, test_size=0.3)

In [133]:
catclass.fit(Xl_train, yl_train)

0:	learn: 0.5266142	total: 47.4ms	remaining: 47.3s
1:	learn: 0.4167228	total: 90.8ms	remaining: 45.3s
2:	learn: 0.3423846	total: 134ms	remaining: 44.5s
3:	learn: 0.2927316	total: 177ms	remaining: 44s
4:	learn: 0.2545434	total: 218ms	remaining: 43.4s
5:	learn: 0.2294725	total: 260ms	remaining: 43.1s
6:	learn: 0.2113647	total: 302ms	remaining: 42.9s
7:	learn: 0.1981064	total: 344ms	remaining: 42.6s
8:	learn: 0.1865253	total: 385ms	remaining: 42.4s
9:	learn: 0.1794322	total: 427ms	remaining: 42.3s
10:	learn: 0.1732272	total: 470ms	remaining: 42.3s
11:	learn: 0.1691910	total: 512ms	remaining: 42.2s
12:	learn: 0.1656684	total: 555ms	remaining: 42.1s
13:	learn: 0.1627485	total: 597ms	remaining: 42s
14:	learn: 0.1603873	total: 640ms	remaining: 42s
15:	learn: 0.1588428	total: 682ms	remaining: 42s
16:	learn: 0.1573075	total: 724ms	remaining: 41.9s
17:	learn: 0.1562691	total: 766ms	remaining: 41.8s
18:	learn: 0.1554581	total: 808ms	remaining: 41.7s
19:	learn: 0.1546399	total: 850ms	remaining: 41

165:	learn: 0.1506650	total: 6.98s	remaining: 35.1s
166:	learn: 0.1506637	total: 7.02s	remaining: 35s
167:	learn: 0.1506634	total: 7.06s	remaining: 35s
168:	learn: 0.1506624	total: 7.1s	remaining: 34.9s
169:	learn: 0.1506616	total: 7.14s	remaining: 34.9s
170:	learn: 0.1506605	total: 7.18s	remaining: 34.8s
171:	learn: 0.1506595	total: 7.22s	remaining: 34.8s
172:	learn: 0.1506589	total: 7.26s	remaining: 34.7s
173:	learn: 0.1506578	total: 7.31s	remaining: 34.7s
174:	learn: 0.1506572	total: 7.35s	remaining: 34.6s
175:	learn: 0.1506571	total: 7.39s	remaining: 34.6s
176:	learn: 0.1506568	total: 7.43s	remaining: 34.6s
177:	learn: 0.1506566	total: 7.47s	remaining: 34.5s
178:	learn: 0.1506550	total: 7.51s	remaining: 34.5s
179:	learn: 0.1506539	total: 7.56s	remaining: 34.4s
180:	learn: 0.1506536	total: 7.6s	remaining: 34.4s
181:	learn: 0.1506529	total: 7.64s	remaining: 34.4s
182:	learn: 0.1506518	total: 7.68s	remaining: 34.3s
183:	learn: 0.1506513	total: 7.72s	remaining: 34.3s
184:	learn: 0.1506

325:	learn: 0.1505338	total: 13.7s	remaining: 28.2s
326:	learn: 0.1505337	total: 13.7s	remaining: 28.2s
327:	learn: 0.1505332	total: 13.7s	remaining: 28.1s
328:	learn: 0.1505330	total: 13.8s	remaining: 28.1s
329:	learn: 0.1505329	total: 13.8s	remaining: 28s
330:	learn: 0.1505327	total: 13.9s	remaining: 28s
331:	learn: 0.1505322	total: 13.9s	remaining: 28s
332:	learn: 0.1505320	total: 13.9s	remaining: 27.9s
333:	learn: 0.1505312	total: 14s	remaining: 27.9s
334:	learn: 0.1505309	total: 14s	remaining: 27.8s
335:	learn: 0.1505301	total: 14.1s	remaining: 27.8s
336:	learn: 0.1505297	total: 14.1s	remaining: 27.7s
337:	learn: 0.1505297	total: 14.1s	remaining: 27.7s
338:	learn: 0.1505295	total: 14.2s	remaining: 27.7s
339:	learn: 0.1505286	total: 14.2s	remaining: 27.6s
340:	learn: 0.1505272	total: 14.3s	remaining: 27.6s
341:	learn: 0.1505272	total: 14.3s	remaining: 27.5s
342:	learn: 0.1505269	total: 14.4s	remaining: 27.5s
343:	learn: 0.1505267	total: 14.4s	remaining: 27.4s
344:	learn: 0.1505265	

485:	learn: 0.1504571	total: 20.4s	remaining: 21.5s
486:	learn: 0.1504571	total: 20.4s	remaining: 21.5s
487:	learn: 0.1504568	total: 20.4s	remaining: 21.4s
488:	learn: 0.1504531	total: 20.5s	remaining: 21.4s
489:	learn: 0.1504519	total: 20.5s	remaining: 21.4s
490:	learn: 0.1504514	total: 20.6s	remaining: 21.3s
491:	learn: 0.1504512	total: 20.6s	remaining: 21.3s
492:	learn: 0.1504512	total: 20.6s	remaining: 21.2s
493:	learn: 0.1504511	total: 20.7s	remaining: 21.2s
494:	learn: 0.1504511	total: 20.7s	remaining: 21.1s
495:	learn: 0.1504486	total: 20.8s	remaining: 21.1s
496:	learn: 0.1504480	total: 20.8s	remaining: 21.1s
497:	learn: 0.1504478	total: 20.9s	remaining: 21s
498:	learn: 0.1504470	total: 20.9s	remaining: 21s
499:	learn: 0.1504470	total: 20.9s	remaining: 20.9s
500:	learn: 0.1504470	total: 21s	remaining: 20.9s
501:	learn: 0.1504470	total: 21s	remaining: 20.9s
502:	learn: 0.1504468	total: 21.1s	remaining: 20.8s
503:	learn: 0.1504466	total: 21.1s	remaining: 20.8s
504:	learn: 0.150444

645:	learn: 0.1503975	total: 27.1s	remaining: 14.9s
646:	learn: 0.1503974	total: 27.2s	remaining: 14.8s
647:	learn: 0.1503974	total: 27.2s	remaining: 14.8s
648:	learn: 0.1503967	total: 27.2s	remaining: 14.7s
649:	learn: 0.1503965	total: 27.3s	remaining: 14.7s
650:	learn: 0.1503965	total: 27.3s	remaining: 14.6s
651:	learn: 0.1503960	total: 27.4s	remaining: 14.6s
652:	learn: 0.1503959	total: 27.4s	remaining: 14.6s
653:	learn: 0.1503956	total: 27.4s	remaining: 14.5s
654:	learn: 0.1503955	total: 27.5s	remaining: 14.5s
655:	learn: 0.1503955	total: 27.5s	remaining: 14.4s
656:	learn: 0.1503954	total: 27.6s	remaining: 14.4s
657:	learn: 0.1503954	total: 27.6s	remaining: 14.4s
658:	learn: 0.1503953	total: 27.7s	remaining: 14.3s
659:	learn: 0.1503953	total: 27.7s	remaining: 14.3s
660:	learn: 0.1503946	total: 27.7s	remaining: 14.2s
661:	learn: 0.1503946	total: 27.8s	remaining: 14.2s
662:	learn: 0.1503946	total: 27.8s	remaining: 14.1s
663:	learn: 0.1503945	total: 27.9s	remaining: 14.1s
664:	learn: 

805:	learn: 0.1503637	total: 33.8s	remaining: 8.15s
806:	learn: 0.1503637	total: 33.9s	remaining: 8.1s
807:	learn: 0.1503637	total: 33.9s	remaining: 8.06s
808:	learn: 0.1503636	total: 34s	remaining: 8.02s
809:	learn: 0.1503633	total: 34s	remaining: 7.98s
810:	learn: 0.1503632	total: 34.1s	remaining: 7.94s
811:	learn: 0.1503623	total: 34.1s	remaining: 7.89s
812:	learn: 0.1503623	total: 34.1s	remaining: 7.85s
813:	learn: 0.1503623	total: 34.2s	remaining: 7.81s
814:	learn: 0.1503622	total: 34.2s	remaining: 7.77s
815:	learn: 0.1503613	total: 34.3s	remaining: 7.73s
816:	learn: 0.1503613	total: 34.3s	remaining: 7.68s
817:	learn: 0.1503612	total: 34.4s	remaining: 7.64s
818:	learn: 0.1503612	total: 34.4s	remaining: 7.6s
819:	learn: 0.1503611	total: 34.4s	remaining: 7.56s
820:	learn: 0.1503611	total: 34.5s	remaining: 7.52s
821:	learn: 0.1503609	total: 34.5s	remaining: 7.47s
822:	learn: 0.1503609	total: 34.6s	remaining: 7.43s
823:	learn: 0.1503608	total: 34.6s	remaining: 7.39s
824:	learn: 0.1503

965:	learn: 0.1503324	total: 40.6s	remaining: 1.43s
966:	learn: 0.1503324	total: 40.7s	remaining: 1.39s
967:	learn: 0.1503324	total: 40.7s	remaining: 1.34s
968:	learn: 0.1503322	total: 40.7s	remaining: 1.3s
969:	learn: 0.1503321	total: 40.8s	remaining: 1.26s
970:	learn: 0.1503303	total: 40.8s	remaining: 1.22s
971:	learn: 0.1503289	total: 40.9s	remaining: 1.18s
972:	learn: 0.1503289	total: 40.9s	remaining: 1.14s
973:	learn: 0.1503280	total: 40.9s	remaining: 1.09s
974:	learn: 0.1503278	total: 41s	remaining: 1.05s
975:	learn: 0.1503278	total: 41s	remaining: 1.01s
976:	learn: 0.1503278	total: 41.1s	remaining: 967ms
977:	learn: 0.1503277	total: 41.1s	remaining: 925ms
978:	learn: 0.1503276	total: 41.2s	remaining: 883ms
979:	learn: 0.1503276	total: 41.2s	remaining: 841ms
980:	learn: 0.1503274	total: 41.2s	remaining: 799ms
981:	learn: 0.1503273	total: 41.3s	remaining: 757ms
982:	learn: 0.1503273	total: 41.3s	remaining: 715ms
983:	learn: 0.1503268	total: 41.4s	remaining: 673ms
984:	learn: 0.150

In [134]:
accuracy_score(yl_test, catclass.predict(Xl_test))

0.9424314765236214

In [135]:
f1_score(yl_test, catclass.predict(Xl_test))

0.6171700127414856

**Now with class balance**

In [144]:
catclass = CatBoostClassifier(class_weights=[1.,11.])

In [145]:
catclass.fit(Xl_train, yl_train)

0:	learn: 0.6069645	total: 43ms	remaining: 43s
1:	learn: 0.5407681	total: 87.3ms	remaining: 43.6s
2:	learn: 0.4928593	total: 131ms	remaining: 43.5s
3:	learn: 0.4554595	total: 175ms	remaining: 43.6s
4:	learn: 0.4286256	total: 217ms	remaining: 43.2s
5:	learn: 0.4065806	total: 260ms	remaining: 43.1s
6:	learn: 0.3898065	total: 303ms	remaining: 43s
7:	learn: 0.3772058	total: 346ms	remaining: 42.9s
8:	learn: 0.3666192	total: 387ms	remaining: 42.7s
9:	learn: 0.3587055	total: 429ms	remaining: 42.5s
10:	learn: 0.3525242	total: 475ms	remaining: 42.7s
11:	learn: 0.3467478	total: 516ms	remaining: 42.5s
12:	learn: 0.3427872	total: 558ms	remaining: 42.4s
13:	learn: 0.3393824	total: 600ms	remaining: 42.3s
14:	learn: 0.3363787	total: 644ms	remaining: 42.3s
15:	learn: 0.3345946	total: 687ms	remaining: 42.2s
16:	learn: 0.3328860	total: 729ms	remaining: 42.1s
17:	learn: 0.3314575	total: 771ms	remaining: 42.1s
18:	learn: 0.3303007	total: 813ms	remaining: 42s
19:	learn: 0.3293409	total: 855ms	remaining: 41

165:	learn: 0.3221027	total: 7.11s	remaining: 35.7s
166:	learn: 0.3220941	total: 7.15s	remaining: 35.7s
167:	learn: 0.3220894	total: 7.19s	remaining: 35.6s
168:	learn: 0.3220876	total: 7.24s	remaining: 35.6s
169:	learn: 0.3220841	total: 7.28s	remaining: 35.5s
170:	learn: 0.3220805	total: 7.32s	remaining: 35.5s
171:	learn: 0.3220727	total: 7.36s	remaining: 35.4s
172:	learn: 0.3220639	total: 7.41s	remaining: 35.4s
173:	learn: 0.3220584	total: 7.45s	remaining: 35.4s
174:	learn: 0.3220520	total: 7.49s	remaining: 35.3s
175:	learn: 0.3220260	total: 7.54s	remaining: 35.3s
176:	learn: 0.3220158	total: 7.58s	remaining: 35.3s
177:	learn: 0.3220121	total: 7.62s	remaining: 35.2s
178:	learn: 0.3220075	total: 7.66s	remaining: 35.2s
179:	learn: 0.3219900	total: 7.71s	remaining: 35.1s
180:	learn: 0.3219683	total: 7.75s	remaining: 35.1s
181:	learn: 0.3219626	total: 7.8s	remaining: 35s
182:	learn: 0.3219590	total: 7.84s	remaining: 35s
183:	learn: 0.3219518	total: 7.88s	remaining: 35s
184:	learn: 0.32194

325:	learn: 0.3206871	total: 14s	remaining: 29s
326:	learn: 0.3206848	total: 14.1s	remaining: 28.9s
327:	learn: 0.3206836	total: 14.1s	remaining: 28.9s
328:	learn: 0.3206807	total: 14.1s	remaining: 28.8s
329:	learn: 0.3206640	total: 14.2s	remaining: 28.8s
330:	learn: 0.3206623	total: 14.2s	remaining: 28.8s
331:	learn: 0.3206564	total: 14.3s	remaining: 28.7s
332:	learn: 0.3206342	total: 14.3s	remaining: 28.7s
333:	learn: 0.3206302	total: 14.4s	remaining: 28.6s
334:	learn: 0.3206288	total: 14.4s	remaining: 28.6s
335:	learn: 0.3206191	total: 14.4s	remaining: 28.5s
336:	learn: 0.3206140	total: 14.5s	remaining: 28.5s
337:	learn: 0.3206122	total: 14.5s	remaining: 28.5s
338:	learn: 0.3206061	total: 14.6s	remaining: 28.4s
339:	learn: 0.3206039	total: 14.6s	remaining: 28.4s
340:	learn: 0.3205998	total: 14.7s	remaining: 28.3s
341:	learn: 0.3205866	total: 14.7s	remaining: 28.3s
342:	learn: 0.3205839	total: 14.7s	remaining: 28.2s
343:	learn: 0.3205829	total: 14.8s	remaining: 28.2s
344:	learn: 0.32

485:	learn: 0.3196395	total: 20.9s	remaining: 22.1s
486:	learn: 0.3196370	total: 21s	remaining: 22.1s
487:	learn: 0.3196317	total: 21s	remaining: 22s
488:	learn: 0.3196108	total: 21.1s	remaining: 22s
489:	learn: 0.3196059	total: 21.1s	remaining: 22s
490:	learn: 0.3196013	total: 21.1s	remaining: 21.9s
491:	learn: 0.3195990	total: 21.2s	remaining: 21.9s
492:	learn: 0.3195858	total: 21.2s	remaining: 21.8s
493:	learn: 0.3195707	total: 21.3s	remaining: 21.8s
494:	learn: 0.3195702	total: 21.3s	remaining: 21.7s
495:	learn: 0.3195650	total: 21.4s	remaining: 21.7s
496:	learn: 0.3195641	total: 21.4s	remaining: 21.7s
497:	learn: 0.3195507	total: 21.4s	remaining: 21.6s
498:	learn: 0.3195407	total: 21.5s	remaining: 21.6s
499:	learn: 0.3195399	total: 21.5s	remaining: 21.5s
500:	learn: 0.3195326	total: 21.6s	remaining: 21.5s
501:	learn: 0.3195308	total: 21.6s	remaining: 21.4s
502:	learn: 0.3195300	total: 21.7s	remaining: 21.4s
503:	learn: 0.3195124	total: 21.7s	remaining: 21.4s
504:	learn: 0.3195111	

645:	learn: 0.3187552	total: 27.8s	remaining: 15.3s
646:	learn: 0.3187378	total: 27.9s	remaining: 15.2s
647:	learn: 0.3187352	total: 27.9s	remaining: 15.2s
648:	learn: 0.3187342	total: 28s	remaining: 15.1s
649:	learn: 0.3187303	total: 28s	remaining: 15.1s
650:	learn: 0.3187125	total: 28.1s	remaining: 15s
651:	learn: 0.3187089	total: 28.1s	remaining: 15s
652:	learn: 0.3187077	total: 28.1s	remaining: 15s
653:	learn: 0.3187010	total: 28.2s	remaining: 14.9s
654:	learn: 0.3186813	total: 28.2s	remaining: 14.9s
655:	learn: 0.3186807	total: 28.3s	remaining: 14.8s
656:	learn: 0.3186789	total: 28.3s	remaining: 14.8s
657:	learn: 0.3186782	total: 28.4s	remaining: 14.7s
658:	learn: 0.3186778	total: 28.4s	remaining: 14.7s
659:	learn: 0.3186763	total: 28.4s	remaining: 14.7s
660:	learn: 0.3186755	total: 28.5s	remaining: 14.6s
661:	learn: 0.3186748	total: 28.5s	remaining: 14.6s
662:	learn: 0.3186744	total: 28.6s	remaining: 14.5s
663:	learn: 0.3186738	total: 28.6s	remaining: 14.5s
664:	learn: 0.3186711	

805:	learn: 0.3182302	total: 34.7s	remaining: 8.36s
806:	learn: 0.3182290	total: 34.8s	remaining: 8.31s
807:	learn: 0.3182167	total: 34.8s	remaining: 8.27s
808:	learn: 0.3182159	total: 34.9s	remaining: 8.23s
809:	learn: 0.3182147	total: 34.9s	remaining: 8.19s
810:	learn: 0.3182124	total: 34.9s	remaining: 8.14s
811:	learn: 0.3182107	total: 35s	remaining: 8.1s
812:	learn: 0.3182096	total: 35s	remaining: 8.06s
813:	learn: 0.3182087	total: 35.1s	remaining: 8.01s
814:	learn: 0.3182072	total: 35.1s	remaining: 7.97s
815:	learn: 0.3182068	total: 35.2s	remaining: 7.93s
816:	learn: 0.3182052	total: 35.2s	remaining: 7.88s
817:	learn: 0.3181927	total: 35.2s	remaining: 7.84s
818:	learn: 0.3181924	total: 35.3s	remaining: 7.8s
819:	learn: 0.3181847	total: 35.3s	remaining: 7.75s
820:	learn: 0.3181738	total: 35.4s	remaining: 7.71s
821:	learn: 0.3181735	total: 35.4s	remaining: 7.67s
822:	learn: 0.3181733	total: 35.5s	remaining: 7.62s
823:	learn: 0.3181727	total: 35.5s	remaining: 7.58s
824:	learn: 0.3181

965:	learn: 0.3176678	total: 41.6s	remaining: 1.46s
966:	learn: 0.3176635	total: 41.6s	remaining: 1.42s
967:	learn: 0.3176631	total: 41.7s	remaining: 1.38s
968:	learn: 0.3176624	total: 41.7s	remaining: 1.33s
969:	learn: 0.3176616	total: 41.8s	remaining: 1.29s
970:	learn: 0.3176613	total: 41.8s	remaining: 1.25s
971:	learn: 0.3176584	total: 41.9s	remaining: 1.21s
972:	learn: 0.3176582	total: 41.9s	remaining: 1.16s
973:	learn: 0.3176513	total: 42s	remaining: 1.12s
974:	learn: 0.3176502	total: 42s	remaining: 1.08s
975:	learn: 0.3176442	total: 42s	remaining: 1.03s
976:	learn: 0.3176396	total: 42.1s	remaining: 991ms
977:	learn: 0.3176368	total: 42.1s	remaining: 948ms
978:	learn: 0.3176224	total: 42.2s	remaining: 905ms
979:	learn: 0.3176193	total: 42.2s	remaining: 862ms
980:	learn: 0.3176189	total: 42.3s	remaining: 818ms
981:	learn: 0.3176185	total: 42.3s	remaining: 775ms
982:	learn: 0.3176086	total: 42.3s	remaining: 732ms
983:	learn: 0.3176048	total: 42.4s	remaining: 689ms
984:	learn: 0.3176

In [146]:
accuracy_score(yl_test, catclass.predict(Xl_test))

0.8948573085666645

In [147]:
f1_score(yl_test, catclass.predict(Xl_test))

0.5597762456094706